In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import os
import matplotlib.pyplot as plt

# 1. Load pre-trained Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

pipe = pipe.to("cuda")

# 2. Create folder to save synthetic dataset
output_dir = "synthetic_images"
os.makedirs(output_dir, exist_ok=True)

# 3. Input prompts (minimum 5)
prompts = [
    "A futuristic city at sunset",
    "An astronaut riding a horse in space",
    "A cyberpunk street at night",
    "A robot studying in a classroom",
    "A peaceful mountain landscape"
]

# 4. Generate images
generated_images = []

for i, prompt in enumerate(prompts):
    image = pipe(prompt).images[0]
    image_path = f"{output_dir}/image_{i+1}.png"
    image.save(image_path)
    generated_images.append(image)

# 5. Display generated images
plt.figure(figsize=(12, 6))

for i, img in enumerate(generated_images):
    plt.subplot(2, 3, i + 1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Image {i+1}")

plt.show()


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import os

# Load pre-trained Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)
pipe = pipe.to("cuda")

# Root dataset folder
dataset_dir = "cat_species_dataset"
os.makedirs(dataset_dir, exist_ok=True)

# List of cat species (40+)
cat_species = [
    "Abyssinian cat",
    "American Bobtail cat",
    "American Curl cat",
    "American Shorthair cat",
    "American Wirehair cat",
    "Balinese cat",
    "Bengal cat",
    "Birman cat",
    "Bombay cat",
    "British Shorthair cat",
    "Burmese cat",
    "Chartreux cat",
    "Cornish Rex cat",
    "Devon Rex cat",
    "Egyptian Mau cat",
    "Exotic Shorthair cat",
    "Havana Brown cat",
    "Himalayan cat",
    "Japanese Bobtail cat",
    "Korat cat",
    "LaPerm cat",
    "Maine Coon cat",
    "Manx cat",
    "Norwegian Forest cat",
    "Ocicat cat",
    "Oriental Shorthair cat",
    "Persian cat",
    "Ragdoll cat",
    "Russian Blue cat",
    "Savannah cat",
    "Scottish Fold cat",
    "Selkirk Rex cat",
    "Siamese cat",
    "Siberian cat",
    "Singapura cat",
    "Somali cat",
    "Sphynx cat",
    "Tonkinese cat",
    "Turkish Angora cat",
    "Turkish Van cat"
]

images_per_species = 10

# Generate dataset
for species in cat_species:
    species_folder = os.path.join(dataset_dir, species.replace(" ", "_"))
    os.makedirs(species_folder, exist_ok=True)

    for i in range(images_per_species):
        prompt = f"A high quality realistic photo of a {species}"
        image = pipe(prompt).images[0]
        image.save(f"{species_folder}/{species.replace(' ', '_')}_{i+1}.png")

    print(f"Generated images for: {species}")

print("Dataset generation completed!")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import shutil
import os

source = "/content/cat_species_dataset"
destination = "/content/drive/MyDrive/cat_species_dataset"

# If folder already exists in Drive, delete it first
if os.path.exists(destination):
    shutil.rmtree(destination)

# Copy entire dataset
shutil.copytree(source, destination)

print("✅ Dataset successfully uploaded to Google Drive in structured format!")


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image
import os


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
dataset_path = "/content/cat_species_dataset"

dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

loader = DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
)

class_names = dataset.classes
num_classes = len(class_names)

print("Total classes:", num_classes)


In [ ]:
model = models.efficientnet_b0(pretrained=True)

# Freeze pretrained layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

model = model.to("cuda")


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)


In [ ]:
epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to("cuda"), labels.to("cuda")

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")


In [ ]:
def classify_image(image_path):
    model.eval()

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to("cuda")

    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]


In [ ]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in loader:
        images, labels = images.to("cuda"), labels.to("cuda")

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = (correct / total) * 100
print(f"✅ Model Accuracy: {accuracy:.2f}%")
